In [96]:
import torch
from torch import nn, autograd as ag
import numpy as np
import pdb
import random
import torch.nn.functional as F
from torch.autograd import Variable
import math
import torch.nn.functional as fn

In [97]:
class NoisyLinear(nn.Linear):
    def __init__(self, in_features, out_features, sigma_init=0.017, bias=True):
        super(NoisyLinear, self).__init__(in_features, out_features, bias=bias)
        self.sigma_weight = nn.Parameter(torch.Tensor(out_features, in_features).fill_(sigma_init))
        self.register_buffer("epsilon_weight", torch.zeros(out_features, in_features))
        if bias:
            self.sigma_bias = nn.Parameter(torch.Tensor(out_features).fill_(sigma_init))
            self.register_buffer("epsilon_bias", torch.zeros(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        std = math.sqrt(3 / self.in_features)
        nn.init.uniform(self.weight, -std, std)
        nn.init.uniform(self.bias, -std, std)

    def forward(self, input):
        torch.randn(self.epsilon_weight.size(), out=self.epsilon_weight)
        bias = self.bias
        if bias is not None:
            torch.randn(self.epsilon_bias.size(), out=self.epsilon_bias)
            bias = bias + self.sigma_bias * Variable(self.epsilon_bias)
        return F.linear(input, self.weight + self.sigma_weight * Variable(self.epsilon_weight), bias)


In [98]:
# corr_mat = torch.tensor([[1., 0.2, 0.4],
#                      [0.2, 1., 0.7],
#                      [0.4, 0.7, 1.0]])
# loc = torch.tensor([0.9, 0.5, 0.1])
corr_mat = torch.tensor([[1.00, -0.85, -0.78,  0.68, -0.87,  0.42],
[-0.85, 1.00,  0.79, -0.71,  0.89, -0.43],
[-0.78,  0.79,  1.00, -0.45,  0.66, -0.71],
[0.68, -0.71, -0.45,  1.00, -0.71,  0.09],
[-0.87,  0.89,  0.66, -0.71,  1.00, -0.17],
[0.42, -0.43, -0.71,  0.09, -0.17,  1.00]])
loc = torch.tensor([18.1,  225, 105, 2.76, 3.460, 20.22])

scaling = torch.distributions.Uniform(torch.tensor(0.), torch.tensor(1.))
mvn = torch.distributions.MultivariateNormal(loc, corr_mat )
data = torch.cat([mvn.sample() * scaling.sample() for i in range(100)]).view(100, -1)

from sklearn import datasets
data = torch.from_numpy(datasets.make_blobs(n_features=6, n_samples=100, centers=10, center_box=[0, 1000])[0]).view(100, -1).type(torch.FloatTensor)


In [99]:
def sample(vec):
    return vec + torch.distributions.Normal(0, 0.5).sample_n(len(vec))

In [149]:
class NeuralNet(nn.Module):
    def __init__(self, w = None):
        super(NeuralNet, self).__init__()
        self.embed = nn.Embedding(100, 32)
        if w is not None:
            self.embed.weight = nn.Parameter(w)
        #self.fc1 = NoisyLinear(16, 16) 
        self.fc1 = nn.Linear(32, 16) 
        self.relu1 = nn.ReLU()
        #self.fc2 = NoisyLinear(16, 16)
        self.fc2 = nn.Linear(16, 16)
        self.relu2 = nn.ReLU()
        #self.fc3 = NoisyLinear(16, 6)
        self.fc3 = nn.Linear(16, 6)
    def forward(self, x):
        out = self.embed(x)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

model = NeuralNet()



In [150]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  
loss = torch.nn.MSELoss()

In [152]:
num_epochs = 200
for epoch in range(num_epochs):
    total_loss = 0
    for i in range(len(data)):  
        output = model.forward(torch.tensor([i]))
        if random.random() < .9:
            chosen_action = torch.distributions.Categorical(fn.softmax(output,1)).sample()
        else:
            #chosen_action = torch.argmax(output)
            chosen_action = torch.tensor([random.randint(0, 5)]).type(torch.LongTensor)

        #chosen_action = torch.tensor([random.randint(0, 2)])
        #pdb.set_trace()
        responsible_weight = output.gather(1, chosen_action.view(-1, 1))
        mse_loss = loss(responsible_weight.view(-1), sample(data[i])[chosen_action]) + 0.7 * loss(model.embed(torch.tensor([i])), torch.zeros_like(model.embed(torch.tensor([i]))))
        total_loss += mse_loss
        optimizer.zero_grad()
        mse_loss.backward()
        optimizer.step()

    if epoch % 1== 0:
        embd = model.embed.weight.mean(0).repeat(1, 100).view(100, -1)
        #model.embed.weight = nn.Parameter(embd)
        correct = 0
        for i in range(len(data)):
            if torch.argmax(model.forward(torch.tensor([i]))) == torch.argmax(data[i]):
                correct += 1
#             if torch.argmax(model.forward(torch.tensor([i]))) != torch.argmax(data[i]):
#                 print data[i]
#                 print model.forward(torch.tensor([i]))
#                 print "----------"
#         pdb.set_trace()
        print ('Epoch [{}/{}], Loss: {:.4f}, Acc: {}%' 
                       .format(epoch+1, num_epochs,  total_loss / len(data), float(correct)/len(data)))


Epoch [1/200], Loss: 14072.7188, Acc: 0.53%
Epoch [2/200], Loss: 11022.4990, Acc: 0.53%
Epoch [3/200], Loss: 9254.6064, Acc: 0.52%
Epoch [4/200], Loss: 15682.7734, Acc: 0.45%
Epoch [5/200], Loss: 5963.8955, Acc: 0.47%
Epoch [6/200], Loss: 5970.9004, Acc: 0.5%
Epoch [7/200], Loss: 5103.2007, Acc: 0.53%
Epoch [8/200], Loss: 7536.0850, Acc: 0.53%
Epoch [9/200], Loss: 8667.5508, Acc: 0.5%
Epoch [10/200], Loss: 10445.3291, Acc: 0.53%
Epoch [11/200], Loss: 12857.0322, Acc: 0.55%
Epoch [12/200], Loss: 10103.3369, Acc: 0.56%
Epoch [13/200], Loss: 11088.2373, Acc: 0.57%
Epoch [14/200], Loss: 14248.4912, Acc: 0.56%
Epoch [15/200], Loss: 8987.9688, Acc: 0.56%
Epoch [16/200], Loss: 9018.5400, Acc: 0.56%
Epoch [17/200], Loss: 11773.4121, Acc: 0.56%
Epoch [18/200], Loss: 10582.8340, Acc: 0.56%
Epoch [19/200], Loss: 11043.0117, Acc: 0.56%
Epoch [20/200], Loss: 10980.7930, Acc: 0.55%
Epoch [21/200], Loss: 10308.3047, Acc: 0.53%
Epoch [22/200], Loss: 9843.3477, Acc: 0.53%
Epoch [23/200], Loss: 14603.45

Epoch [186/200], Loss: 5500.6421, Acc: 0.6%
Epoch [187/200], Loss: 6621.1475, Acc: 0.6%
Epoch [188/200], Loss: 4497.7412, Acc: 0.6%
Epoch [189/200], Loss: 7575.1177, Acc: 0.6%
Epoch [190/200], Loss: 4636.3950, Acc: 0.64%
Epoch [191/200], Loss: 6209.1362, Acc: 0.69%
Epoch [192/200], Loss: 5214.1055, Acc: 0.65%
Epoch [193/200], Loss: 9021.6221, Acc: 0.62%
Epoch [194/200], Loss: 3696.4690, Acc: 0.69%
Epoch [195/200], Loss: 11117.6396, Acc: 0.69%
Epoch [196/200], Loss: 5920.7061, Acc: 0.7%
Epoch [197/200], Loss: 7826.8862, Acc: 0.73%
Epoch [198/200], Loss: 6909.7612, Acc: 0.62%
Epoch [199/200], Loss: 13758.5391, Acc: 0.66%
Epoch [200/200], Loss: 12342.5947, Acc: 0.65%


In [153]:
i = random.randint(0, len(data))
print (data[i]), torch.argmax(data[i])
print (model.forward(torch.tensor([i]))), torch.argmax(model.forward(torch.tensor([i])))


tensor([ 897.3497,  246.2585,  832.8798,  780.5245,  184.0292,  516.8893]) tensor(0)
tensor([[ 745.3607,  504.9086,  875.2828,  347.3766,  398.0196,  492.4910]]) tensor(2)


In [154]:
embd = model.embed.weight.mean(0).repeat(1, 100).view(100, -1)
model.embed.weight = nn.Parameter(embd)

In [155]:

model.fc1.requires_grad = False
model.fc2.requires_grad = False
model.fc3.requires_grad = False
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)  
loss = torch.nn.MSELoss()

In [168]:
num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0
    for i in range(len(data)):  
        output = model.forward(torch.tensor([i]))
        chosen_action = torch.distributions.Categorical(fn.softmax(output,1)).sample()
        #chosen_action = torch.argmax(output)

        #chosen_action = torch.tensor([random.randint(0, 2)])
        #pdb.set_trace()
        responsible_weight = output.gather(1, chosen_action.view(-1, 1))
        mse_loss = loss(responsible_weight.view(-1), sample(data[i])[chosen_action])
        total_loss += mse_loss
        optimizer.zero_grad()
        mse_loss.backward()
        optimizer.step()
    if epoch % 1 == 0:
        correct = 0
        for i in range(len(data)):
            if torch.argmax(model.forward(torch.tensor([i]))) == torch.argmax(data[i]):
                correct += 1
#             if torch.argmax(model.forward(torch.tensor([i]))) != torch.argmax(data[i]):
#                 print data[i]
#                 print model.forward(torch.tensor([i]))
#                 print "----------"
#         pdb.set_trace()
        print ('Epoch [{}/{}], Loss: {:.4f}, Acc: {}%' 
                       .format(epoch+1, num_epochs,  total_loss / len(data), float(correct)/len(data)))


Epoch [1/10], Loss: 3888.9941, Acc: 0.46%
Epoch [2/10], Loss: 6521.1162, Acc: 0.46%
Epoch [3/10], Loss: 14658.1953, Acc: 0.47%
Epoch [4/10], Loss: 3618.3181, Acc: 0.47%
Epoch [5/10], Loss: 3006.0510, Acc: 0.47%
Epoch [6/10], Loss: 2979.8181, Acc: 0.47%
Epoch [7/10], Loss: 2513.6782, Acc: 0.44%
Epoch [8/10], Loss: 13323.3047, Acc: 0.45%
Epoch [9/10], Loss: 5533.4487, Acc: 0.47%
Epoch [10/10], Loss: 4660.0049, Acc: 0.47%


In [175]:
i = random.randint(0, len(data))
print (data[i]), torch.argmax(data[i])
print (model.forward(torch.tensor([i]))), torch.argmax(model.forward(torch.tensor([i])))

tensor([  95.6899,  376.7789,  604.5927,  899.2483,  263.2460,  758.4307]) tensor(3)
tensor([[  93.0321,  142.1794,   78.7361,  -19.5789,  170.4666,  105.8602]]) tensor(4)
